# 1. Question or problem definition.
#  เนื่องจากเราอยากจะทำเพลงขึ้นมาและพบว่าปัจจุบันนี้ได้มีแนวเพลงใหม่ ๆ เกิดเป็นจำนวนมาก จึงได้สำรวจว่าการที่เพลงจะมียอดนิยมที่สูงนั้น มีข้อมูลอะไรบ้างที่จำเป็นสำหรับการนำมาแต่งเพลงเพื่อให้มีค่าความนิยมของเพลงสูงขึ้น เช่น จังหวะของเพลง หรือ ความสนุกของเพลง เป็นต้น โดยกลุ่มลูกค้าของเราสามารถเป็นได้ทั้งเจ้าของค่ายเพลง นักแต่งเพลง นักการตลาด นักร้อง ศิลปิน ฯลฯ 


# 2.Acquire training and testing data.

# เป็นการ import library เข้ามาใช้

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set(rc={'figure.figsize':(25,15)})
from IPython.core.interactiveshell import InteractiveShell
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.cluster.hierarchy import linkage, dendrogram
import scipy.cluster.hierarchy as sch
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
import statsmodels.api as sm
from sklearn import preprocessing
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import silhouette_score
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

# เป็นการนำข้อมูลเข้ามาใน notebook

In [ ]:
df = pd.read_csv('/kaggle/input/top-spotify-songs-from-20102019-by-year/top10s.csv', encoding='latin1')
df1 = pd.read_csv('/kaggle/input/top-spotify-songs-from-20102019-by-year/top10s.csv', encoding='latin1')
df.head()
df.shape

# เป็นการเปลี่ยนชื่อตัว column unnamed:0 ให้เป็น id และแสดงค่าออกมาดู

In [ ]:
df = df.rename(columns = {'Unnamed: 0': 'id'})
df.head()

# 3. Wrangle, prepare, cleanse the data.

# เป็นการเช็คว่ามีข้อมูลสูญหายไปรึเปล่า

In [ ]:
df.isnull().any()

# หาผลรวมของข้อมูลที่หายไป

In [ ]:
df.isnull().sum()

In [ ]:
df = df.drop_duplicates()
df.shape

# ดรอปข้อมูลที่ไม่ต้องการใช้ออกไป

In [ ]:
df = df.drop(['id'], axis=1)
df.head()

# เป็นการดูค่าต่าง ๆ ของข้อมูล เช่น Max Min

In [ ]:
df.describe().T

# เป็นการหาค่าข้อมูลที่เยอะที่สุด ของ top genre,artist,title,year

In [ ]:
df['top genre'].value_counts()
df['artist'].value_counts()
df['title'].value_counts()
df['year'].value_counts()

# เป็นการหาชื่อเพลงที่มีคำว่า Company 

In [ ]:
df[df.title == 'Company']

# เป็นการหาค่า Max ของข้อมูล pop, dnce, spch ที่นักร้องแต่ละคนทำได้ โดยเรียงจากค่า pop

In [ ]:
song=df[df["pop"]==df["pop"]].groupby(["artist"]).agg({"pop":'max',"dnce":'max',"spch":'max'}).sort_values(["pop"],ascending=False)
print(song)

# #**เป็นการหานักร้องที่มีเพลง rap ที่มีความนิยมเยอะมากที่สุด**

In [ ]:
raphit=song[(song["pop"]>=90)&(song["spch"]>=30)]
print (raphit)

# ****ดรอปข้อมูลที่ไม่ต้องการใช้ คือ year, title, dB, live, dur

In [ ]:
yearless_df = df.drop(['year', 'title','dB','live','dur'], axis=1)
yearless_df.drop_duplicates()

# เช็ค top genre ว่าเข้าเงื่อนไขไหน ถ้าเข้าเงื่อนไขใด ๆ แล้วให้ replace ค่านั้น ๆ ออกมา
# เพื่อที่จะรวมข้อมูลที่เป็น top genre ที่มีความใกล้เคียงกัน เช่น R&B ไปรวมกับ Pop หรือ Electro ไปรวมกับ EDM เพื่อจัดการข้อมูล

In [ ]:
for i in yearless_df['top genre']:
    if 'pop' in i:
        yearless_df['top genre'] = yearless_df['top genre'].replace(i, 'pop')
        
    elif 'hip hop' in i:
        yearless_df['top genre'] = yearless_df['top genre'].replace(i, 'hip hop')

    elif 'edm' in i:
        yearless_df['top genre'] = yearless_df['top genre'].replace(i, 'edm')

    elif 'r&b' in i:
        yearless_df['top genre'] = yearless_df['top genre'].replace(i, 'pop')

    elif 'latin' in i:
        yearless_df['top genre'] = yearless_df['top genre'].replace(i, 'latin')

    elif 'room' in i:
        yearless_df['top genre'] = yearless_df['top genre'].replace(i, 'room')

    elif 'electro' in i:
        yearless_df['top genre'] = yearless_df['top genre'].replace(i, 'edm')
        
yearless_df['top genre'] = yearless_df['top genre'].replace('chicago rap', 'hip hop')
        
yearless_df["top genre"]

# นับจำนวนของ top genre

In [ ]:
yearless_df['top genre'].value_counts()

In [ ]:
yearless_df

In [ ]:
# genre_df = pd.DataFrame(yearless_df['top genre'].value_counts()).reset_index()
# genre_df.columns = ['top genre','count']
# genre_df['top_genre_modeling'] = genre_df['top genre'] 
# genre_df.loc[genre_df['count']< 4,'top_genre_modeling'] = 'other'
# genre_df = genre_df.drop(['top genre'], axis=1)
# genre_df

# ทำการเช็คจำนวนของแต่ละแนวเพลง ว่าถ้าแนวเพลงไหนมีจำนวนน้อยกว่าหรือเท่ากับ 3 ให้นำมารวมเป็น other

In [ ]:
temp_df = yearless_df
value_counts = temp_df.stack().value_counts() # Entire DataFrame 
to_remove = value_counts[value_counts <= 3].index
temp_df.replace(to_remove, 'other', inplace=True)
temp_df['top genre'].value_counts()
temp_df.head()
temp_df.shape

In [ ]:
yearless_df['top genre'] = temp_df['top genre']
yearless_df[['bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch', 'artist']] = df[['bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch', 'artist']]
yearless_df['top genre'].value_counts()
yearless_df.head()

# เป็นการเทียบข้อมูลค่าต่าง ๆ ของ yearless_df

In [ ]:
sns.pairplot(yearless_df, vars=['bpm','nrgy','dnce','val','acous','spch','pop'],
            hue='top genre'
            
           );

In [ ]:
hit=df[df["pop"]==df["pop"]].groupby(["title"]).agg({"pop":'max',"dnce":'max',"spch":'max',"bpm":'max'}).sort_values(["pop"],ascending=False)
print(hit)

# 4.Analyze, identify patterns, and explore the data.

# ****เช็คว่าปีไหนที่เพลงมีความ popularity มากที่สุด

In [ ]:
pop=df[df["pop"]==df["pop"]].groupby(["year"]).agg({"pop":'max',"dnce":'max',"spch":'max',"bpm":'max'}).sort_values(["pop"],ascending=False)
print(pop)

# แล้วมาพล็อตกราฟเทียบข้อมูลเพื่อสำรวจว่าแนวเพลงไหนที่มียอดนิยมสูงที่สุด

In [ ]:
plot_data =yearless_df.groupby('top genre')['pop'].max()

plot_data.sort_values()[-10:].plot(kind='bar')
plt.title("Popurality Genre")
plt.ylabel("Popurality")
plt.show()

# เป็นการทำ histogram เพื่อแสดงค่าต่าง ๆ ออกมา
# เพื่อดูกราฟแต่ละอันว่ามีค่าประมาณเท่าไหร่

In [ ]:
plt.hist(yearless_df.dB)
plt.xlabel('Loudness dB')
plt.show()
plt.hist(yearless_df.bpm)
plt.xlabel('Beat per minute')
plt.show()
plt.hist(yearless_df.nrgy)
plt.xlabel('Energy')
plt.show()
plt.hist(yearless_df.live)
plt.xlabel('Liveness')
plt.show()
plt.hist(yearless_df.val)
plt.xlabel('Valence')
plt.show()
plt.hist(yearless_df.dur)
plt.xlabel('Durable')
plt.show()
plt.hist(yearless_df.acous)
plt.xlabel('Acousticness')
plt.show()
plt.hist(yearless_df.spch)
plt.xlabel('Speechiness')
plt.show()

# 5.Model, predict and solve the problem.

# สร้างตัวแปรแยกออกมาสองตัว เพื่อนำไปใช้ใน Hierarchical และ K-Means โดยจะทำการดรอปคอลัมท์ออกเพิ่มเติมเนื่องจาก ข้อมูลสามตัวนี้เราไม่ต้องการนำมาใช้

In [ ]:
dfh = yearless_df.drop(['live','dB','dur'], axis=1)
dfk = yearless_df.drop(['live','dB','dur'], axis=1)

# การทำ hierarchical clustering

# สร้างตัวแปร cols เพื่อระบุคอลัมท์ ที่ต้องการใช้

In [ ]:
cols =['bpm','nrgy','dnce','val','acous','spch','pop']

# สร้างตัวแปร pt ขึ้นมาเพื่อทำการแปลงข้อมูลให้อยู่ในรูปของ Gaussian เพื่อให้เป็นตัวเลขและง่ายต่อการนำไปทำ cluster

In [ ]:
pt = preprocessing.PowerTransformer(method='yeo-johnson',standardize=True)
mat = pt.fit_transform(dfh[cols])
mat[:5].round(4)

In [ ]:
X=pd.DataFrame(mat, columns=cols)
X.head()

# เป็นการทำ Dendrogram เพื่อสำรวจข้อมูลจำนวนกลุ่มของ cluster

In [ ]:
fig, ax=plt.subplots(figsize=(100,20))
dg=sch.dendrogram(sch.linkage(X,method='ward'),ax=ax,labels=dfh['top genre'].values)

# สร้าตัวแปร hc เพื่อเก็บค่า cluster ของ hierarchical ที่มีจำนวน cluster = 8 และใช้ linkage แบบ ward

In [ ]:
hc=AgglomerativeClustering(n_clusters=8,linkage='ward')
hc

# เป็นการ fit model ของ hc

In [ ]:
hc.fit(X)

In [ ]:
hc.labels_

# เป็นการเพิ่ม cluster เข้าไปใน column ของ dfh

In [ ]:
dfh['cluster']=hc.labels_
dfh.head()

# แสดงค่า 10 ตัวอย่าง

In [ ]:
dfh.head(10)

# เช็คค่าต่าง ๆ ของแต่ละ cluster จากข้อมูล Hierarchical

In [ ]:
dfh.groupby('cluster').agg(['count','mean','median']).T

# นำข้อมูลของแต่ละ cluster อย่างละ 3 ตัวมาแสดงค่าเรียงกัน

In [ ]:
dfh.groupby('cluster').head(3).sort_values('cluster')

# เป็นการพล็อตกราฟไวโอลิน เพื่อดูค่าที่ชัดเจนมากขึ้นของแต่ละ cluster ในแต่ละข้อมูล

In [ ]:
cols =['bpm','nrgy','dnce','val','acous','spch','pop']
fig,ax = plt.subplots(nrows=4,ncols=2,figsize=(20,9))
ax=ax.ravel()
for i, col in enumerate(cols):
    sns.violinplot(x='cluster',y=col,data=dfh,ax=ax[i])

In [ ]:
data = dfh.iloc[:, 3:5].values

# ดูการจัดกลุ่มของ cluster ทั้ง 8 กลุ่ม 

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(data[:,0], data[:,1], c=dfh.cluster, cmap='rainbow')

# แสดงจำนวนแนวเพลงในแต่ละ cluster

In [ ]:
pd.crosstab(dfh['top genre'],hc.labels_)

# การทำ k-mean clustering และ ทำการprediction

In [ ]:
X = dfk[['bpm','nrgy','dnce','val','acous','spch','pop']]

# เป็นการค่า K ที่ใช้เพื่อหาจำนวน cluster ที่จะใช้ใน K-Means

In [ ]:
sum_of_squared_distances = []
K = range(1,20)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(X)
    sum_of_squared_distances.append(km.inertia_)
    
ax = sns.lineplot(x=K, y = sum_of_squared_distances)
ax.set(xlabel='K', ylabel='sum of squared distances', title='Elbow graph')

# จากกราฟทำให้ทราบว่าเราควรจะใช้จำนวน cluster เท่าไหร่ 

# ใน model นี้ใช้ cluster = 8 

In [ ]:
model = KMeans(n_clusters=8)
model

# นำตัวแปร model มา fit ข้อมูล

In [ ]:
model.fit(X)

# เป็นการเอาตัวแปร model มาหาจุดกึ่งกลาง cluster

In [ ]:
model.cluster_centers_

In [ ]:
model.labels_

# แสดงค่าของ top genre

In [ ]:
dfk['top genre'].values

# เป็นการเพิ่ม cluster เข้าไปใน column ของ dfk

In [ ]:
dfk['cluster']=model.labels_

# เป็นการแสดงข้อมูลจำนวนแนวเพลงในแต่ละ cluster

In [ ]:
pd.crosstab(dfk['top genre'],model.labels_)

# เป็นการแสดงค่า 10 ตัว

In [ ]:
dfk.head(10)

# เป็นการนำข้อมูลออกมา 3 ตัว เพื่อแสดงตัวอย่างข้อมูลแต่ละ cluster

In [ ]:
dfk.groupby('cluster').head(3).sort_values('cluster')

# เป็นใส่ตัวข้อมูลลงไปเพื่อให้ model ทำการ cluster เพื่อให้ทราบผลลัพธ์ว่าเมื่อทำการ cluster แล้ว หน้าตาของการจัดกลุ่มจะออกมาเป็นรูปแบบไหน 

In [ ]:
model.predict([
    [65,88,70,50,3,13,75],
    [120,50,70,70,2,15,75]
])

# 6. Visualize, report, and present the problem solving steps and final solution.

# ข้างล่างเป็นการ plot กราฟเพื่อแสดงค่าต่าง ๆ ของแต่ละ cluster

In [ ]:
sns.catplot(x="cluster",y="bpm",data=dfh)
plt.xlabel('cluster H')
sns.catplot(x="cluster",y="bpm",data=dfk)
plt.xlabel('cluster K')

In [ ]:
sns.catplot(x="cluster",y="nrgy",data=dfh)
plt.xlabel('cluster H')
sns.catplot(x="cluster",y="nrgy",data=dfk)
plt.xlabel('cluster K')

In [ ]:
sns.catplot(x="cluster",y="dnce",data=dfh)
plt.xlabel('cluster H')
sns.catplot(x="cluster",y="dnce",data=dfk)
plt.xlabel('cluster K')

In [ ]:
sns.catplot(x="cluster",y="val",data=dfh)
plt.xlabel('cluster H')
sns.catplot(x="cluster",y="val",data=dfk)
plt.xlabel('cluster K')

In [ ]:
sns.catplot(x="cluster",y="acous",data=dfh)
plt.xlabel('cluster H')
sns.catplot(x="cluster",y="acous",data=dfk)
plt.xlabel('cluster K')

In [ ]:
sns.catplot(x="cluster",y="spch",data=dfh)
plt.xlabel('cluster H')
sns.catplot(x="cluster",y="spch",data=dfk)
plt.xlabel('cluster K')

In [ ]:
sns.catplot(x="cluster",y="pop",data=dfh)
plt.xlabel('cluster H')
sns.catplot(x="cluster",y="pop",data=dfk)
plt.xlabel('cluster K')

# สรุปได้ว่าการทำ Model ทั้งสองตัวนั้นพบว่า K-Means มีการจัดกลุ่มข้อมูลเกาะกลุ่มกว่า Hierarchical และพบว่าข้อมูลแต่ละ cluster มีการแบ่งระดับของแต่ละ cluster ค่อนข้างชัดเจน และ เราสามารถนำมาดูได้ว่า cluster ที่มีค่าความ pop ค่อนข้างสูง สามารถเอาตัวแปรไหนมาใช้เพื่อเพิ่มความนิยมให้เพลงได้ ซึ่งการทำ clustering นี้อาจจะทำให้กลุ่มคนที่สนใจเกี่ยวกับเพลงสามารถนำไปวิเคราะห์เพื่อสร้างเพลงได้